# Data Augmentation & Balancing Pipeline

---

## 1. Zielsetzung dieses Notebooks
Wie in der explorativen Datenanalyse (EDA) festgestellt wurde, weist der Datensatz ein massives **Klassenungleichgewicht (Class Imbalance)** auf. Häufige Klassen wie *Transverse Displaced* dominieren, während komplexe Frakturen wie *Segmental* oder *Linear* stark unterrepräsentiert sind.

Dieses Notebook implementiert eine **Offline-Augmentation-Pipeline**, um dieses Ungleichgewicht vor dem Training zu korrigieren. Ziel ist es, die Anzahl der Trainingsbilder für seltene Klassen synthetisch zu erhöhen (Upsampling), um dem Modell eine statistisch ausgewogene Lernbasis zu bieten.

## 2. Technische Umsetzung
Die Augmentation erfolgt mithilfe der Bibliothek **Albumentations**, die speziell für Computer Vision Aufgaben optimiert ist und sicherstellt, dass Bounding-Box-Koordinaten bei geometrischen Transformationen korrekt mit angepasst werden.

**Verwendete Transformationen:**
Um die Varianz der Daten zu erhöhen und das Modell robuster zu machen, werden folgende Techniken kombiniert:
*   **Geometrisch:** Horizontales Spiegeln, Rotation, Skalierung, Verschiebung.
*   **Pixel-Level:** Anpassung von Helligkeit/Kontrast, Gaußsches Rauschen, Unschärfe (Blur).

## 3. Ablauf
1.  **Konfiguration:** Definition eines `augmentation_plan`, der festlegt, welche Klasse mit welchem Multiplikator vervielfacht werden soll (z. B. *Segmental* x20).
2.  **Pipeline-Definition:** Erstellung der `A.Compose` Pipeline inkl. Bounding-Box-Handling.
3.  **Generierung:** Das Skript iteriert über die Originalbilder, wendet die Transformationen an und speichert die neuen Bilder sowie die angepassten Labels (mit Präfix `aug_`) physisch im Trainingsordner.
4.  **Bereinigung (Optional):** Eine Hilfsfunktion am Ende erlaubt das schnelle Löschen aller generierten Dateien, um den Ursprungszustand wiederherzustellen.

---


In [ ]:
# Importieren der notwendigen Bibliotheken
import cv2  # OpenCV: Zum Lesen, Schreiben und Bearbeiten von Bildern
import albumentations as A  # Albumentations: Bibliothek für die Bild-Augmentation
import os  # os: Für die Interaktion mit dem Betriebssystem, z.B. um auf Dateien und Ordner zuzugreifen
import random  # random: Für zufällige Operationen (wird von Albumentations intern genutzt)

# ==============================================================================
# 1. KONFIGURATION
# In diesem Abschnitt werden alle wichtigen Parameter definiert.
# ==============================================================================

# --- Pfade zu den Trainingsdaten ---
# Hier geben wir an, wo sich die Bilder und die zugehörigen Label-Dateien befinden.
train_img_path = os.path.join('BoneFracturesDetection', 'train', 'images')
train_lbl_path = os.path.join('BoneFracturesDetection', 'train', 'labels')

# --- Definition der Klassennamen ---
# Die Reihenfolge ist entscheidend, da der Index der Liste der Klassen-ID entspricht (z.B. 'Comminuted' == ID 0).
class_names = ['Comminuted', 'Greenstick', 'Healthy', 'Linear', 'Oblique Displaced', 'Oblique', 'Segmental', 'Spiral', 'Transverse Displaced', 'Transverse']

# --- Augmentations-Plan ---
# Wir definieren hier, für welche (seltenen) Klassen wir neue Daten erzeugen wollen
# und 'wie viele' neue Bilder für jedes existierende Originalbild dieser Klasse erstellt werden sollen.
# Ziel ist es, die Anzahl der Instanzen der seltenen Klassen an die der häufigeren Klassen anzugleichen.
augmentation_plan = {
    'Segmental': 20,  # Z.B. aus 18 Originalbildern werden ca. 18 * 20 = 360 
    'Linear': 20,     # von 21 -> 420
    'Oblique': 10,    # von 48 -> 480
    'Healthy': 8,     # von 54 -> 432
    'Spiral': 8,      # von 66 -> 528
    'Greenstick': 5,  # von 81 -> 405
    'Transverse': 2,
    # Klassen, die hier nicht aufgeführt sind (z.B. 'Transverse Displaced'), werden nicht augmentiert.
}


# ==============================================================================
# 2. DEFINITION DER AUGMENTATIONS-PIPELINE
# Hier legen wir fest, welche zufälligen Veränderungen auf die Bilder angewendet werden sollen.
# ==============================================================================

# A.Compose erstellt eine Pipeline von Augmentations-Schritten.
# Albumentations sorgt dafür, dass die Bounding Boxes korrekt mit-transformiert werden.
transform = A.Compose([
    # Spiegelt das Bild horizontal mit einer Wahrscheinlichkeit von 50%.
    A.HorizontalFlip(p=0.5),
    
    # Ändert zufällig Helligkeit und Kontrast des Bildes.
    A.RandomBrightnessContrast(p=0.3),
    
    # Führt zufällig eine Verschiebung, Skalierung und Rotation des Bildes durch.
    A.ShiftScaleRotate(shift_limit=0.06, scale_limit=0.1, rotate_limit=15, p=0.7),
    
    # Fügt dem Bild zufälliges Gaußsches Rauschen hinzu, um es robuster zu machen.
    A.GaussNoise(p=0.2),
    
    # 'OneOf' wendet nur eine der folgenden Transformationen an.
    A.OneOf([
        A.Blur(blur_limit=3, p=0.5),      # Macht das Bild leicht unscharf.
        A.MotionBlur(blur_limit=3, p=0.5), # Simuliert eine Bewegungsunschärfe.
    ], p=0.5), # Diese 'OneOf'-Box wird mit einer Wahrscheinlichkeit von 50% ausgeführt.

# bbox_params sorgt dafür, dass die Bounding-Box-Koordinaten bei jeder Transformation korrekt mit angepasst werden. 'yolo' gibt das Format an (class_id, x_center, y_center, width, height).
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


# ==============================================================================
# 3. DURCHFÜHRUNG DER AUGMENTATION
# ==============================================================================

print("--- Starte die Erstellung neuer augmentierter Daten ---")

# --- Zuerst sammeln wir alle Bilder und ordnen sie ihrer Klasse zu ---
# Wir erstellen ein Dictionary, um zu speichern, welche Bilddatei zu welcher Klasse gehört.
image_class_map = {}
for img_file in os.listdir(train_img_path):
    # Wir nehmen nur Originalbilder, keine bereits augmentierten.
    if img_file.startswith('aug_'):
        continue

    # Finde die passende Label-Datei zum Bild.
    label_file = img_file.replace('.jpg', '.txt').replace('.jpeg', '.txt')
    label_path = os.path.join(train_lbl_path, label_file)

    # Stelle sicher, dass eine Label-Datei existiert.
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            try:
                # Lese die erste Zeile der Label-Datei, um die Klasse zu bestimmen.
                # Dies ist eine Vereinfachung; sie funktioniert gut, wenn Bilder meist nur Objekte einer Klasse enthalten.
                first_line = f.readline()
                if first_line: # Stelle sicher, dass die Datei nicht leer ist
                    class_id = int(float(first_line.split()[0])) # Robustes Parsen: erst zu float, dann zu int
                    image_class_map[img_file] = class_names[class_id]
            except (IndexError, ValueError):
                # Ignoriere leere oder fehlerhafte Label-Dateien.
                print(f"Warnung: Konnte Label-Datei {label_file} nicht verarbeiten.")
                continue

print(f"-> {len(image_class_map)} Originalbilder und ihre Klassen wurden eingelesen.")

# --- Jetzt führen wir die eigentliche Augmentation durch ---
# Wir gehen durch jedes gefundene Originalbild.
for img_file, class_name in image_class_map.items():
    # Prüfe, ob die Klasse dieses Bildes in unserem Augmentations-Plan enthalten ist.
    if class_name in augmentation_plan:
        # Hole die Anzahl der zu erzeugenden neuen Bilder.
        num_augmentations = augmentation_plan[class_name]
        
        # Lade das Originalbild und die zugehörigen Bounding-Box-Daten.
        img_path = os.path.join(train_img_path, img_file)
        label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.jpeg', '.txt')
        
        # Lade das Bild mit OpenCV.
        image = cv2.imread(img_path)
        # Konvertiere das Bild vom BGR-Format (OpenCV-Standard) in das RGB-Format (Albumentations-Standard).
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Lese alle Bounding Boxes und Klassen-Labels aus der Label-Datei.
        bboxes = []
        class_labels = []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_labels.append(int(float(parts[0])))
                    bboxes.append([float(p) for p in parts[1:]])
        
        # Wenn keine Boxen gefunden wurden, überspringe das Bild.
        if not bboxes:
            continue

        # Erzeuge die festgelegte Anzahl an neuen, augmentierten Bildern.
        for i in range(num_augmentations):
            # Wende die oben definierte Transformations-Pipeline an.
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            
            # Erstelle einen neuen, eindeutigen Dateinamen für das augmentierte Bild.
            new_img_filename = f"aug_{class_name}_{i}_{img_file}"
            
            # Speichere das neue Bild im Trainingsordner. Konvertiere es zurück ins BGR-Format.
            cv2.imwrite(os.path.join(train_img_path, new_img_filename), cv2.cvtColor(augmented['image'], cv2.COLOR_RGB2BGR))
            
            # Erstelle den passenden Dateinamen für das neue Label.
            new_label_filename = new_img_filename.replace('.jpg', '.txt').replace('.jpeg', '.txt')
            
            # Speichere die transformierten Bounding Boxes im YOLO-Format.
            with open(os.path.join(train_lbl_path, new_label_filename), 'w') as f:
                for box, label in zip(augmented['bboxes'], augmented['class_labels']):
                    # Stelle sicher, dass die Klassen-ID als ganze Zahl (Integer) geschrieben wird.
                    f.write(f"{int(label)} {' '.join(map(str, box))}\n")

print(f"\n--- Augmentation abgeschlossen! ---")
print(f"Überprüfen Sie die Ordner '{train_img_path}' und '{train_lbl_path}' auf neue Dateien mit dem Präfix 'aug_'.")
print("Führe  nun die EDA-Zelle erneut aus, um die neue, ausbalancierte Klassenverteilung zu sehen.")

In [2]:
# Löschen der augemntierten Bilder
import os

train_img_path = os.path.join('BoneFracturesDetection', 'train', 'images')
train_lbl_path = os.path.join('BoneFracturesDetection', 'train', 'labels')

def cleanup_augmented_files(path, prefix='aug_'):
    count = 0
    for filename in os.listdir(path):
        if filename.startswith(prefix):
            os.remove(os.path.join(path, filename))
            count += 1
    return count

print("Lösche alte Augmentations-Dateien...")
img_deleted = cleanup_augmented_files(train_img_path)
lbl_deleted = cleanup_augmented_files(train_lbl_path)
print(f"{img_deleted} augmentierte Bilder gelöscht.")
print(f"{lbl_deleted} augmentierte Labels gelöscht.")
print("Bereinigung abgeschlossen.")


Lösche alte Augmentations-Dateien...
2655 augmentierte Bilder gelöscht.
2655 augmentierte Labels gelöscht.
Bereinigung abgeschlossen.
